In [39]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import torch
import os
import time
import math
import sys
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from torch.autograd import grad
from torch.nn import Parameter
cuda = torch.cuda.is_available()
import pickle as pk
import torch.optim as optim
import torch.nn as nn
from collections import OrderedDict
from torchvision.transforms import transforms


import fish_models
import robofish.io

fishes = 4
in_channels_global = 1
out_channels_global = 1

torch.cuda.is_available()

False

In [40]:
# ResNet uses blocks
class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResNetBlock, self).__init__()
        self.kernel_size = 3

        # structure
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=self.kernel_size, stride=1, padding=1)
        self.relu1 = nn.LeakyReLU()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv2 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=self.kernel_size, stride=1, padding=1)
        self.relu2 = nn.LeakyReLU()
        self.bn2 = nn.BatchNorm2d(in_channels)
        self.conv3 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=self.kernel_size, stride=1, padding=1)
        self.relu3 = nn.LeakyReLU()
        self.bn3 = nn.BatchNorm2d(in_channels)
        self.conv4 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels * 2, kernel_size=self.kernel_size, stride=2, padding=(32 - 15))
        self.relu4 = nn.LeakyReLU()
        self.bn4 = nn.BatchNorm2d(in_channels * 2)

        # 1x1 conv filters can be used to change the dimensionality in the filter space.
        self.identity_upsample = nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=out_channels * 2, kernel_size=1, stride=1),
                                          nn.BatchNorm2d(in_channels * 2))
        #self.identity_downsample = nn.Sequential(nn.Conv2d(in_channels=out_channels * 2, out_channels=in_channels, kernel_size=1, stride=1), nn.BatchNorm2d(in_channels))

    def forward(self, x):
        # we use this block multipy times
        # DEBUG
        #print(f'x forward start: {x.shape}')

        # save first identity
        identity1 = x.clone()

        x = self.conv1(x)
        x = self.relu1(x)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.bn2(x)

        # sum point
        x = x + identity1

        # save second identity and change size to be able to summurize in the next sum point
        identity_upsampled = self.identity_upsample(x)
        # DEBUG
        #print(f'identity_upsample : {identity_upsampled.shape}')

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.bn3(x)
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.bn4(x)

        # DEBUG
        #print(f'x for last sum point: {x.shape}')  

        # sum point
        x = x + identity_upsampled
        return x

block = ResNetBlock(in_channels=in_channels_global, out_channels=out_channels_global)
block

ResNetBlock(
  (conv1): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
  (relu1): LeakyReLU(negative_slope=0.01)
  (bn1): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): LeakyReLU(negative_slope=0.01)
  (bn2): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): LeakyReLU(negative_slope=0.01)
  (bn3): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(1, 2, kernel_size=(3, 3), stride=(2, 2), padding=(17, 17))
  (relu4): LeakyReLU(negative_slope=0.01)
  (bn4): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (identity_upsample): Sequential(
    (0): Conv2d(1, 2, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [5]:
# # 2d: [batch_size, num_features (aka: C * H * W)]
# use for nn.Linear() input.

class ResNet(nn.Module): 
    # layers is the list telling us how much to reuse the block in each block
    def __init__(self, block, N, image_channel, num_classes, batch_size=64):
        """ResNet is a short name for Residual Network

        Args:
            speed_bins: The array with the float borders between speed bins
            turn_bins: The array with the float borders between turn bins
        """
        self.speed_bins = speed_bins
        self.turn_bins = turn_bins
        
        super(ResNet, self).__init__()
        self.in_channels = in_channels_global
        self.kernel_size = 3
        self.start_identity = None
        self.linear_size = 808920
        self.batch_size = batch_size
        self.batch_small = 2
        self.N = N

        # Begin
        self.conv1 = nn.Conv2d(in_channels=image_channel, out_channels=1, kernel_size=self.kernel_size, stride=1, padding=1)
        self.relu1 = nn.LeakyReLU()
        self.bn1 = nn.BatchNorm2d(1)

        # ResNet Layers
        # repeat 3 times for i in range(2)
        # out_channel is equal to 16 * 2 * i => 16*2^0, 16*2^1, 16*2^2 => 16, 32 and 64 out_channels
        self.layer1 = self.make_layer(block, self.N, out_channels=2)
        self.layer2 = self.make_layer(block, self.N, out_channels=4)
        self.layer3 = self.make_layer(block, self.N, out_channels=8)

        # End
        self.fc_10 = nn.Linear(in_features=self.linear_size, out_features=num_classes)

    def forward(self, x):
    
        # Begin 
        x = self.conv1(x)
        x = self.bn1(x)

        # ResNet Layers forward step
        # loops inside every layer
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        #print('\nExit looped structure')

        # End
        #print(f'\nafter layers structure: {x.shape}')    
        if x.shape[0] == self.batch_size:
            x = x.view(self.batch_size, -1)
        else:
            x = x.view(x.shape[0], -1)
        #print(f'\nafter view(): {x.shape}')

        x = self.fc_10(x)
        #print(f'after fc_10: {x.shape}')

        return x
    
    def make_layer(self, block, num_residual_blocks, out_channels):
        identity_downsample = None
        layers = []

        print(f'Extern loop START')
        layers.append(block(self.in_channels, out_channels))
        self.in_channels = out_channels * 2

          # intern loops for layers and skip connections
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, out_channels))
            print(f'Inner layer {n} is created!')

        print(f'Extern loop END')
        return nn.Sequential(*layers)


In [37]:
class ResNetFishModel(fish_models.gym_interface.AbstractModel):
    def __init__(self):
        self.deep_model = ResNet(ResNetBlock, 1, image_channel=in_channels_global, num_classes=out_channels_global, batch_size=64)
        
    def choose_action(self, view: np.ndarray):

        #action = self.predict_action(view)
        
        #returns speed, turn
        #return action[0][0], action[0][1]
        return 10, 2*np.pi
    
    def predict_action(self, view: np.ndarray):
        pass
        #clustermodel.predict([view]) returns the index for list clustermodel.cluster_centers_
        #prediction = clustermodel.cluster_centers_[clustermodel.predict([view])][0]
        #choice = random.sample(list(clusters[str(prediction)]), 1)
        #return choice
   

    def train(self, train_loader, optimizer, criterion, max_epoch):
       
        #inputs = torch.from_numpy(train_loader[:]["views"])
        #labels = torch.from_numpy(train_loader[:]["actions"])
        losses = []
        batch_total = len(train_loader)
        
        for epoch in range(max_epoch):  # loop over the dataset multiple times
            samples_total = 0
            samples_correct = 0
            samples_correct_test = 0

            running_loss = 0.0
            
            #for i in inputs:
            for batch_idx, batch in enumerate(train_loader):
                print(batch)
                # get the inputs; data is a list of [inputs, labels]
                inputs = torch.from_numpy(batch["views"][:]) 
                labels = torch.from_numpy(batch[:]["actions"])  

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                output = self.deep_model(inputs)

                # DEBUG
                #print(f'Input shape: {inputs.shape}')
                #print(f'Output shape: {output.shape}')
                #print(f'Labels: {labels}')
                #print(f'Labels shape: {labels.shape}')
                #print(f'\nIndex: {batch_idx}, len of batch: {len(batch[0])}')

                loss = criterion(output, labels)
                loss.backward()
                optimizer.step()

                yhat = torch.argmax(output, dim=1)

                samples_total += len(labels)
                samples_correct += torch.sum(yhat == labels)
                losses.append(loss.item())

                # print statistics
                if i % 50 == 0:
                    acc = float(samples_correct) / float(samples_total)

                    sys.stdout.write(f'\repoch: {epoch} / {max_epoch} Step: {i}/{batch_total}, Loss: {loss.item():.6f}, Acc: {acc:.2%}')

        print('\nFinished Training')
        return losses
            
    
model = ResNetFishModel()

Extern loop START
Extern loop END
Extern loop START
Extern loop END
Extern loop START
Extern loop END


In [8]:
raycast = fish_models.gym_interface.Raycast(
            n_wall_raycasts=5,
            n_fish_bins=4,
            fov_angle_fish_bins=np.pi,
            fov_angle_wall_raycasts=np.pi,
            world_bounds=([-50, -50], [50, 50]),
        )


In [9]:
data_folder = Path("data/live_female_female/train")

dset = fish_models.datasets.io_dataset.IoDataset(
    data_folder,
    raycast,
    output_strings=["poses", "actions", "views"],
    reduce_dim=2,
    max_files=1,
)

  0%|          | 0/1 [00:00<?, ?it/s]

Loading data from 1 files.


  0%|          | 0/1 [00:00<?, ?it/s]

Calculating views from 1 files.


100%|██████████| 1/1 [00:01<00:00,  1.91s/it]

Status of IoDataset:
The first 3 dimensions are reduced from (1, 2, 8990) to (17980)
poses	(17980, 3):	consisting of x, y, orientation.
actions	(17978, 2):	consisting of speed [cm/s] and turn [rad/s].
views	(17978, 9):	4 fish_bins and 5 wall ray casts.



In [30]:
from torch.utils.data import DataLoader
batch_size = 64

train_loader = DataLoader(dset, batch_size=batch_size)

In [31]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.deep_model.parameters(), lr=0.001, momentum=0.9)

In [38]:
start = time.time()
losses = model.train(train_loader, optimizer, criterion, max_epoch=1)
end = time.time()
print(f'\nTraining took {end - start}s')


{'poses': tensor([[-48.6194,  38.1500,   1.4628],
        [-48.5948,  38.2514,   1.3988],
        [-48.5645,  38.4874,   1.4609],
        [-48.5549,  38.6357,   1.4995],
        [-48.5430,  38.8079,   1.5349],
        [-48.5063,  39.0114,   1.4520],
        [-48.4366,  39.1252,   1.4953],
        [-48.4251,  39.2198,   1.5306],
        [-48.4041,  39.2914,   1.5123],
        [-48.3832,  39.3629,   1.4940],
        [-48.3638,  39.4892,   1.3755],
        [-48.3226,  39.6210,   1.4793],
        [-48.3137,  39.7291,   1.5180],
        [-48.2907,  39.8828,   1.5709],
        [-48.2561,  39.9191,   1.4997],
        [-48.2011,  39.9441,   1.6296],
        [-48.1992,  40.0733,   1.4960],
        [-48.2029,  40.2312,   1.4969],
        [-48.2044,  40.3369,   1.4865],
        [-48.1417,  40.4586,   1.4904],
        [-48.0785,  40.5385,   1.4899],
        [-48.0822,  40.5346,   1.6360],
        [-48.0799,  40.6510,   1.4920],
        [-48.0675,  40.7422,   1.5309],
        [-48.0288,  40.7576,  

TypeError: expected np.ndarray (got Tensor)